In [1]:
from google.colab import drive
import cv2
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
drive.mount('/content/gdrive')
import os
import pickle
from sklearn.model_selection import GridSearchCV

#neccessary imports for performing CNN extraction.
#!wget http://users.sussex.ac.uk/~nq28/bvlc_alexnet.prototxt -O bvlc_alexnet.prototxt
#!wget http://dl.caffe.berkeleyvision.org/bvlc_alexnet.caffemodel -O bvlc_alexnet.caffemodel

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## **Pre-data preperation**

In [0]:
# Root path for the animal images.
root_path = "gdrive/My Drive/Computer Vision/Coursework/JPEGImages/" 

# Listing all the elements within the root path images.
animal_images = os.listdir(root_path)

# Training images label names.
training_labels = ['antelope','grizzly+bear','killer+whale','beaver','dalmatian','horse','german+shepherd','blue+whale','siamese+cat','skunk','mole','tiger','moose'
                    ,'spider+monkey','elephant','gorilla','ox','fox','sheep','hamster','squirrel','rhinoceros','rabbit','bat','giraffe','wolf','chihuahua','weasel'
                    ,'otter','buffalo','zebra','deer','bobcat','lion','mouse','polar+bear','collie','walrus','cow','dolphin']

# Dictionary containing the cnn features for the training images.
training_data = {}

# Testing images label names.
test_classes = ['chimpanzee','giant+panda','leopard','persian+cat','pig','hippopotamus','humpback+whale','raccoon','rat','seal']

# Dictionary containing the cnn features for the testing images.
testing_data = {}

# Listing all the elements within the root path images.
animal_images = os.listdir(root_path)

In [0]:
# Dictionary that will contain 1/3 of the training image blobs. 
training_data1 = {}
# CNN 2/3 of images.
training_data2 = {}
# CNN 3/3 of images.
training_data3 = {}

## **Loading the blobs for each image containing the training label. (divided into 3 parts)**

In [0]:
# Loading each training image, and acquiring each blob for the cnn feature extraction.
# Divided the loading process into 1/3s for computational ease. 
# 1/3
for i,classe in enumerate(training_labels):
  print('Processing class: ',i,classe)
  class_paths = os.listdir(root_path+classe)
  training_data[classe] = []
  class_paths_subsample = class_paths[0:int(len(class_paths)/3)]
  for file in class_paths_subsample:
    if file != '.DS_Store':
      img = cv2.imread(root_path+classe+ "/" +file)
      resized = cv2.resize(img,(224, 224), interpolation = cv2.INTER_CUBIC)
      blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
      training_data[classe].append(blob)

In [0]:
# 2/3
for i,classe in enumerate(training_labels):
  print('Processing class: ',i,classe)
  class_paths = os.listdir(root_path+classe)
  training_data3[classe] = []
  class_paths_subsample = class_paths[int(len(class_paths)/3):int(len(class_paths))-int(len(class_paths)/3)]
  for file in class_paths_subsample:
    if file != '.DS_Store':
      img = cv2.imread(root_path+classe+ "/" +file)
      resized = cv2.resize(img,(224, 224), interpolation = cv2.INTER_CUBIC)
      blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
      training_data3[classe].append(blob)

In [0]:
# 3/3
for i,classe in enumerate(training_labels):
  print('Processing class: ',i,classe)
  class_paths = os.listdir(root_path+classe)
  training_data3[classe] = []
  class_paths_subsample = class_paths[int(len(class_paths))-int(len(class_paths)/3):len(class_paths)]
  for file in class_paths_subsample:
    if file != '.DS_Store':
      img = cv2.imread(root_path+classe+ "/" +file)
      resized = cv2.resize(img,(224, 224), interpolation = cv2.INTER_CUBIC)
      blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
      training_data3[classe].append(blob)

## **CNN feature extract method - training & testing**

In [0]:
# Output: a dictionary containing the training animal labels as the Key 
# and a 4096x1 dimensional array containing the pretrained AlexNet fc7 layer features extracted. 
# Input: The CNN blobs of each training image.
def cnn_extract(blob_dict):
  cnn_features = {}
  net = cv2.dnn.readNetFromCaffe("./bvlc_alexnet.prototxt", "./bvlc_alexnet.caffemodel")
  for i,animal in enumerate(training_labels):
    print("processing:", i, animal)
    cnn_features[animal] = []
    for img in cnn_blobs[animal]:
      net.setInput(img)
      features_img = net.forward("fc7")
      cnn_features[animal].append(features_img)
  return cnn_features 

In [0]:
cnn_features = cnn_extract(cnn_blobs)

In [0]:
# Output: a dictionary containing the testing animal labels as the Key 
# and a 4096x1 dimensional array containing the pretrained AlexNet fc7 layer features extracted. 
# Input: The CNN blobs of each testing image.
def cnn_extract_test(blob_dict):
  cnn_features = {}
  net = cv2.dnn.readNetFromCaffe("./bvlc_alexnet.prototxt", "./bvlc_alexnet.caffemodel")
  for i,animal in enumerate(test_classes):
    print("processing:", i, animal)
    cnn_features[animal] = []
    for img in test_blob[animal]:
      net.setInput(img)
      features_img = net.forward("fc7")
      cnn_features[animal].append(features_img)
  return cnn_features 

In [0]:
test_cnn_features = cnn_extract_test(test_blob)

In [0]:
# Serizalizing the test CNN features extracted from cnn_extract_test method. 
with open('gdrive/My Drive/Computer Vision/Coursework/CNN/test_cnn_featuresp.pickle', 'rb') as handle1:
   test_cnn_features = pickle.load(handle1)

In [0]:
# Serizalizing the train CNN features extracted from cnn_extract method. 
with open('gdrive/My Drive/Computer Vision/Coursework/CNN/train_cnn_features.pickle', 'rb') as handle1:
   train_cnn_features = pickle.load(handle1)

In [0]:
len(test_cnn_features['chimpanzee'])

200

In [0]:
# Serializing the the test feature blobs acquired from the cnn_extract method.
with open('gdrive/My Drive/Computer Vision/Coursework/CNN/test_blob.pickle', 'rb') as handle1:
   test_blob = pickle.load(handle1)

## **Preliminary text files for training**

In [0]:
# Variable containing the trainclass.txt file.
train_class = np.loadtxt('trainclasses.txt', delimiter='\t', usecols=[0], dtype=np.str)  

# Variable containing the testclasses.txt file.
test_class = np.loadtxt('testclasses.txt', delimiter='\t',usecols=[0], dtype=np.str) 

# Reading the predicate binary text file (correlation between animal classes and its attributes)
M = np.loadtxt('predicate-matrix-binary.txt')

# Variable containing the list of attributes. 
predicates = np.loadtxt('predicates.txt',delimiter='\t',usecols=[1], dtype=np.str)

# Variable containing the entire list of classes of animals.
classes = np.loadtxt('classes.txt', delimiter='\t', usecols=[1], dtype=np.str)  

## **Removing the test classes within the predicate for training**

In [0]:
# predicate matrix without the test class animals.
new_M_matrix = np.delete(M, (24,38,14,5,41,13,17,33,23,34), axis = 0)

## **Preperation for creating classifiers**

In [0]:
#preparing the X input value for the classifier training. 
black_list = []
for animal in train_class:
  for feature in train_cnn_features[animal]:
    black_list.append(feature.ravel())
feature_matrix = np.array(black_list)

## **Mapping the predicates to each row** minus testing

In [0]:
# Dictionary containing as Key the individual 85 attributes
# and as Value its corresponding column within the predicate matrix
# mapping only the test animal classes. 
train_pred_dict = {}
column_list = []
for i in range(0,85):
  column_list.append(new_M_matrix[:,i])
  train_pred_dict[predicates[i]] = column_list[i]

In [0]:
# Dictionary containing as Key the individual 85 attributes
# and as Value its corresponding column within the predicate matrix
# mapping all animal classes. 
pred_map = []
pred_dict = {}
for i in range(0,85):
  pred_map.append(M[:,i])
  pred_dict[predicates[i]] = pred_map[i]

## **Building the 1 CNN classifier**

In [0]:
#primary imports
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

# Data split for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV

## *Train_attribute_models*

In [0]:
# Method takes the feature matrix as input and outputs 85 attribute classifiers trained from the CNN features. 
def train_attribute_models(features):
  models = {}
  for att in predicates: 
    y = []
    att_output = train_pred_dict[att]
    for i,animal in enumerate(training_labels):
      if att_output[i] == 1:
        positive = np.ones((len(train_cnn_features[animal]),1), dtype=int)
        for pos_val in positive:
          y.append(pos_val)
      else:
        negative = np.zeros((len(train_cnn_features[animal]),1), dtype=int)
        for neg_val in negative:
          y.append(neg_val)
    X = features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.66, random_state = 42)
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm)
    clf.fit(X_train,y_train)
    print(att,"classifier trained")
    score = clf.score(X_test,y_test)
    print("The",att,"classifier score is : ",score*100)
    models[att] = clf
  return models

In [16]:
classifiers_3 = train_attribute_models(feature_matrix)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: Co

black classifier trained
The black classifier score is :  77.23525681674064


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: Co

white classifier trained
The white classifier score is :  78.5288522511097


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


...

solitary classifier trained
The solitary classifier score is :  75.73874445149016


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: Co

nestspot classifier trained
The nestspot classifier score is :  81.64870006341154


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: Co

domestic classifier trained
The domestic classifier score is :  81.39505389980977


In [0]:
#pickling classifiers
with open('pickled_clf3.pickle', 'wb') as handle:
  pickle.dump(classifiers_3, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('gdrive/My Drive/Computer Vision/Coursework/CNN/pickled_clf1.pickle', 'rb') as handle2:
  pickled_clf1 = pickle.load(handle2)

## **Classification**

In [0]:
# The method will return a dictionary containing the one of each 85 attributes as key 
# and a list of probabilities of the corresponding attributes being present in the images.
# The method will take the 85 classifiers as input.
def compute_attribute_probs(classifiers):
  prob_dict = {}
  for att,clf in classifiers.items():
    prob_list = []
    for animal,img_list in test_cnn_features.items():
      for img in img_list:
        y_proba = clf.predict_proba(img)
        prob_list.append(y_proba)
    prob_dict[att] = prob_list
  return prob_dict

In [0]:
probs = compute_attribute_probs(classifiers_3)

In [0]:
# Serializing the probability dictionary
with open('gdrive/My Drive/Computer Vision/Coursework/CNN/pickled_probs.pickle', 'rb') as handle2:
  probs = pickle.load(handle2)

In [0]:
# Dictionary containing as Key as the attributes and the value as the animal
# outputs whether the attribute is present or not (1 or 0)
predicate_dict = {}
for attribute, predicates in pred_dict.items():
  predicate_dict[attribute]= {}
  for i,animal in enumerate(classes):
    predicate_dict[attribute][animal] = pred_dict[attribute][i]

In [0]:
# Method will return a dictonary where the Key is the test animal classes and the value as the probabilities of the class being in each image.
# The method will take the test cnn features as input. 
def compute_class_probs(test_features,att_probability):
  prob_dict = {}
  total_prob_mult = 1
  for animal in test_features.keys():
    total_prob_mult = 1
    prob_list = []
    for i in range(2471):
      total_prob_mult = 1
      for att in predicates:
        if predicate_dict[att][animal] == 1:
          total_prob_mult = total_prob_mult * probs[att][i][0][1]
        else:
          total_prob_mult = total_prob_mult * probs[att][i][0][0]
      prob_list.append(total_prob_mult)
    prob_dict[animal] = prob_list
  return prob_dict

In [0]:
pre_prob = compute_class_probs(test_cnn_features,probs)

In [0]:
# Generating a list containing each animal name by its corresponding image lengths.  
chimpanzee_list = ['chimpanzee'] * 200
giantpanda_list = ['giant+panda'] * 250
leopard_list = ['leopard'] * 200
persiancat_list = ['persian+cat'] * 200
pig_list = ['pig'] * 270 
hippopotamus_list = ['hippopotamus'] * 300
humpbackwhale_list = ['humpback+whale'] * 300
raccoon_list = ['raccoon'] * 200
rat_list = ['rat'] * 251
seal_list = ['seal'] * 300
animal_list = []
animal_list.extend(chimpanzee_list)
animal_list.extend(giantpanda_list)
animal_list.extend(leopard_list)
animal_list.extend(persiancat_list)
animal_list.extend(pig_list)
animal_list.extend(hippopotamus_list)
animal_list.extend(humpbackwhale_list)
animal_list.extend(raccoon_list)
animal_list.extend(rat_list)
animal_list.extend(seal_list)

In [0]:
# The method will take as input the animal test classes and 
# output a single real number denoting the overall accuracy of the CNN classifer.
def compute_accuracy(test_classes):
  prob_dict = {}
  max_dict = {}
  counter = 0
  for i in range(2471):
    prob_dict = {}
    for animal in test_classes:
      prob_dict[animal] = pre_prob[animal][i]
    if max(prob_dict, key = lambda k: prob_dict[k]) == animal_list[i]:
      counter += 1
  result = (counter/2471) * 100
  return result

In [0]:
accuracy = compute_accuracy(pre_prob)

In [45]:
print(accuracy)

33.10400647511129
